<a href="https://colab.research.google.com/github/MachineSaver/MachineSaver/blob/main/Machine_Saver_Web_Portal_API_Integration_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Saver Web Portal Integration Examples

[Read the API Documentation](https://www.bipom.com/documents/WiPOM/WRTU%20Web%20Portal%20API.pdf)

# Get Your Authorization Token

In [1]:
import requests

# Define the login URL
url = "https://api.nanowipom.com/v1/Login"

# Set the login data
data = {
    'grant_type': 'password',
    'username': 'MachineSaverDemo',  # Replace with actual username
    'password': '!MachineSaverDemo123'   # Replace with actual password
}

# Send a POST request to login and get the access token
response = requests.post(url, data=data)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    token_data = response.json()
    access_token = token_data.get('access_token')
    print("Access Token:", access_token)
else:
    print(f"Failed to login: {response.status_code} - {response.text}")


Access Token: JrRtVk5PZhPGpLE72f6ba6xC9bmZlZJrUU9o-Tsu6yUxT0M9amTFAkA51yK7k09KVgMvMIx64YOlZAIyo2m7ZLYrvGVPKi6cCIoqPirDfopw1rVlfqj8aLPFKG8XYssS_VzygiauurLk0KVdmPaSoLOr6YI4n7hQck98xWAl4W6BG3qqRrZy3G4jmQPu7KHT--oOYJGYCpEYIf3GUGkOhq1gv8sIIqsJoCME2t2tMhMiZZHbHzEgIJ67WaG8pbUXH4LxnqIvNknR5nn4aiirO68gy-VIDXvtjE7y-IEqRDcgWyL5W6wPDPLxxGHrRI4wQaPHoZeWtVdxKDaUs79lCxR0c_bHkpwiKkKQGOYJvVnHu1Th4wEoxONUsXsliMCi7x1exg


# Get Accounts List

In [ ]:
# Define the Accounts List URL
url = "https://api.nanowipom.com/v1/AccountsList"

# Set the Authorization headers with the Bearer token
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send a GET request to retrieve the accounts list
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    accounts_data = response.json()  # Parse the JSON response
    print("Accounts List:", accounts_data)
else:
    print(f"Failed to fetch accounts: {response.status_code} - {response.text}")

# Get Device List

In [3]:
# Define the Devices List URL
url = "https://api.nanowipom.com/v1/DevicesList"

# Send a GET request to retrieve the devices list
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    devices_list = response.json()
    print("Devices List:", devices_list)
else:
    print(f"Failed to fetch devices list: {response.status_code} - {response.text}")


Devices List: [{'Id': 198, 'AccountId': 441, 'Serial': '05DA0034-38375043-43195446', 'IsActive': True, 'IPAddress': '50.28.221.152', 'CreateAt': '2016-02-02T11:15:45.732', 'CreatedbyUser': 29, 'IsCGVtbCom': False, 'DeviceName': 'ABB VM'}, {'Id': 206, 'AccountId': 418, 'Serial': '05DB0034-38375043-43192344', 'IsActive': True, 'IPAddress': '50.28.220.148', 'CreateAt': '2016-05-24T21:21:52.476', 'CreatedbyUser': 85, 'IsCGVtbCom': False, 'DeviceName': 'Evapco #1'}, {'Id': 208, 'AccountId': 441, 'Serial': '05DE0034-38375043-43195544', 'IsActive': True, 'IPAddress': '50.28.221.163', 'CreateAt': '2016-05-26T22:28:28.203', 'CreatedbyUser': 86, 'IsCGVtbCom': False, 'DeviceName': 'Advanced Aeration #1'}, {'Id': 234, 'AccountId': 441, 'Serial': '05D90034-38375043-43193943', 'IsActive': True, 'IPAddress': '10.10.1.4', 'CreateAt': '2016-10-28T23:09:18.683', 'CreatedbyUser': 29, 'IsCGVtbCom': False, 'DeviceName': 'EuroVolt'}, {'Id': 261, 'AccountId': 441, 'Serial': '05D80034-38375043-43195543', 'IsA

# Get Sensor List Using Gateway ID

In [6]:
# Replace 'device_id' with the actual device ID
device_id = 1229  # Example device ID

# Define the Sensor Data URL
url = f"https://api.nanowipom.com/v1/SensorsList?deviceId={device_id}"

# Send a GET request to retrieve the sensor data
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    sensors_data = response.json()
    print("Sensors Data:", sensors_data)
else:
    print(f"Failed to fetch sensor data: {response.status_code} - {response.text}")


Failed to fetch sensor data: 404 - {
  "Message": "No HTTP resource was found that matches the request URI 'https://api.nanowipom.com/v1/SensorsList?deviceId=1229'.",
  "MessageDetail": "No type was found that matches the controller named 'SensorsList'."
}


# Get Data Tags Using Device ID

In [7]:
# Define the Tags List URL with the device ID
device_id = 1229
url = f"https://api.nanowipom.com/v1/TagsList?deviceId={device_id}"

# Set the Authorization header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send a GET request to retrieve the tag list for the specified device
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    tag_list = response.json()
    print("Tag List:", tag_list)
else:
    print(f"Failed to fetch tag list: {response.status_code} - {response.text}")

Tag List: [{'Id': 1133749, 'IsActive': True, 'DeviceId': 1229, 'Name': '8C1F642113000010 - Acceleration Axis 1', 'Type': 'DS', 'Address': 0, 'VirtualAddress': 0, 'Units1': 'g', 'Units2': '', 'LogPeriod': 0, 'MBRtu': 0, 'MBType': None, 'MBAddress': 0, 'MBValueType': None, 'MBValueByteOrder': None, 'IsVMapEnabled': False, 'IsBMapEnabled': False, 'VMapPresent': False, 'BMapPresent': False, 'MaxLoggedAt': None, 'RecordsNumber': 0}, {'Id': 1133750, 'IsActive': True, 'DeviceId': 1229, 'Name': '8C1F642113000010 - Acceleration Axis 2', 'Type': 'DS', 'Address': 0, 'VirtualAddress': 0, 'Units1': 'g', 'Units2': '', 'LogPeriod': 0, 'MBRtu': 0, 'MBType': None, 'MBAddress': 0, 'MBValueType': None, 'MBValueByteOrder': None, 'IsVMapEnabled': False, 'IsBMapEnabled': False, 'VMapPresent': False, 'BMapPresent': False, 'MaxLoggedAt': None, 'RecordsNumber': 0}, {'Id': 1133751, 'IsActive': True, 'DeviceId': 1229, 'Name': '8C1F642113000010 - Acceleration Axis 3', 'Type': 'DS', 'Address': 0, 'VirtualAddress':

# Get Tag Information

In [8]:
# Define the Tag Info URL with the tag ID
tag_id = 1133752
url = f"https://api.nanowipom.com/v1/TagInfo?Id={tag_id}"

# Set the Authorization header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send a GET request to retrieve the tag information
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    tag_info = response.json()
    print("Tag Info:", tag_info)
else:
    print(f"Failed to fetch tag info: {response.status_code} - {response.text}")

Tag Info: {'Id': 1133752, 'IsActive': True, 'DeviceId': 1229, 'Name': '8C1F642113000010 - Temperature', 'Type': 'HR', 'Address': 40200, 'VirtualAddress': 0, 'Units1': 'C', 'Units2': '', 'LogPeriod': 60, 'MBRtu': 0, 'MBType': None, 'MBAddress': 0, 'MBValueType': None, 'MBValueByteOrder': None, 'IsVMapEnabled': False, 'IsBMapEnabled': False, 'VMapPresent': False, 'BMapPresent': False, 'MaxLoggedAt': '2024-10-24T13:10:04.000', 'RecordsNumber': 13700}


# Get Tag Data From a Specific Time

In [9]:
# Define the Tag Data URL with the tag ID
tag_id = 1133752
url = f"https://api.nanowipom.com/v1/TagData?tagId={tag_id}"

# Optional parameters: specify the time range for the data you want to retrieve
params = {
    'startTime': '09-01-2024 00:00:00',  # Specify the start time (MM-dd-yyyy hh:mm:ss)
    'endTime': '10-15-2024 01:00:00'     # Specify the end time (MM-dd-yyyy hh:mm:ss)
}

# Set the Authorization header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send a GET request to retrieve the tag data for the specified tag ID and time range
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    tag_data = response.json()
    print("Tag Data:", tag_data)
else:
    print(f"Failed to fetch tag data: {response.status_code} - {response.text}")


Tag Data: [{'Id': 831275937, 'LogId': 1727736841, 'LoggedAt': '2024-10-04T16:35:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextValue': None}, {'Id': 831276055, 'LogId': 1727736857, 'LoggedAt': '2024-10-04T16:36:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextValue': None}, {'Id': 831276288, 'LogId': 1727736889, 'LoggedAt': '2024-10-04T16:38:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextValue': None}, {'Id': 831276431, 'LogId': 1727736905, 'LoggedAt': '2024-10-04T16:39:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextValue': None}, {'Id': 831276447, 'LogId': 1727736921, 'LoggedAt': '2024-10-04T16:40:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextValue': None}, {'Id': 831276659, 'LogId': 1727736937, 'LoggedAt': '2024-10-04T16:41:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextValue': None}, {'Id': 831276849, 'LogId': 1727736969, 'LoggedAt': '2024-10-04T16:43:50.000', 'RawValue': 2.802597e-44, 'CalcValue': 20.0, 'TextVal

# Get List of Data Snapshots

In [10]:
# Define the Data Snapshots List URL with the device ID
device_id = 1229  # Replace with the actual device ID
url = f"https://api.nanowipom.com/v1/DatasnapshotsList?deviceId={device_id}"

# Set the Authorization header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send a GET request to retrieve the data snapshot list for the specified device
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    snapshots_list = response.json()
    print("Data Snapshots List:", snapshots_list)
else:
    print(f"Failed to fetch data snapshot list: {response.status_code} - {response.text}")

Data Snapshots List: [{'Id': 5445, 'DeviceId': 1229, 'TagId': 1133749, 'Name': 'Acceleration Axis 1', 'XUnits': 'us', 'YUnits': 'g', 'Timestamp': '08/13/2024 14:19:27', 'XData': 'double[252]', 'YData': 'double[252]'}, {'Id': 5446, 'DeviceId': 1229, 'TagId': 1133750, 'Name': 'Acceleration Axis 2', 'XUnits': 'us', 'YUnits': 'g', 'Timestamp': '08/13/2024 14:19:27', 'XData': 'double[252]', 'YData': 'double[252]'}, {'Id': 5447, 'DeviceId': 1229, 'TagId': 1133751, 'Name': 'Acceleration Axis 3', 'XUnits': 'us', 'YUnits': 'g', 'Timestamp': '08/13/2024 14:19:27', 'XData': 'double[252]', 'YData': 'double[252]'}, {'Id': 5712, 'DeviceId': 1229, 'TagId': 1133749, 'Name': 'Acceleration Axis 1', 'XUnits': 'us', 'YUnits': 'g', 'Timestamp': '08/14/2024 09:32:04', 'XData': 'double[252]', 'YData': 'double[252]'}, {'Id': 5713, 'DeviceId': 1229, 'TagId': 1133750, 'Name': 'Acceleration Axis 2', 'XUnits': 'us', 'YUnits': 'g', 'Timestamp': '08/14/2024 09:32:04', 'XData': 'double[252]', 'YData': 'double[252]'}

# Get Data from a Snapshot ID

In [11]:
# Define the Data Snapshot Data URL with the snapshot ID
snapshot_id = 5446  # Replace with the actual snapshot ID
url = f"https://api.nanowipom.com/v1/DatasnapshotData?Id={snapshot_id}"

# Set the Authorization header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Send a GET request to retrieve the data for the specific snapshot
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print("Data Snapshot Data:", data)
else:
    print(f"Failed to fetch data snapshot: {response.status_code} - {response.text}")

Data Snapshot Data: {'Id': 5446, 'DeviceId': 1229, 'TagId': 1133750, 'Name': 'Acceleration Axis 2', 'XUnits': 'us', 'YUnits': 'g', 'Timestamp': '08/13/2024 14:19:27', 'XData': [0.0, 70.0, 140.0, 210.0, 280.0, 350.0, 420.0, 490.0, 560.0, 630.0, 700.0, 770.0, 840.0, 910.0, 980.0, 1050.0, 1120.0, 1190.0, 1260.0, 1330.0, 1400.0, 1470.0, 1540.0, 1610.0, 1680.0, 1750.0, 1820.0, 1890.0, 1960.0, 2030.0, 2100.0, 2170.0, 2240.0, 2310.0, 2380.0, 2450.0, 2520.0, 2590.0, 2660.0, 2730.0, 2800.0, 2870.0, 2940.0, 3010.0, 3080.0, 3150.0, 3220.0, 3290.0, 3360.0, 3430.0, 3500.0, 3570.0, 3640.0, 3710.0, 3780.0, 3850.0, 3920.0, 3990.0, 4060.0, 4130.0, 4200.0, 4270.0, 4340.0, 4410.0, 4480.0, 4550.0, 4620.0, 4690.0, 4760.0, 4830.0, 4900.0, 4970.0, 5040.0, 5110.0, 5180.0, 5250.0, 5320.0, 5390.0, 5460.0, 5530.0, 5600.0, 5670.0, 5740.0, 5810.0, 5880.0, 5950.0, 6020.0, 6090.0, 6160.0, 6230.0, 6300.0, 6370.0, 6440.0, 6510.0, 6580.0, 6650.0, 6720.0, 6790.0, 6860.0, 6930.0, 7000.0, 7070.0, 7140.0, 7210.0, 7280.0, 7

# Plot Snapshot Data

In [12]:
# First, install plotly if not already installed
!pip install plotly

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Set custom theme
def set_plotly_theme():
    pio.templates["draft"] = go.layout.Template(
        layout_annotations=[
            dict(
                name="draft watermark",
                text="Machine Saver Inc.",
                textangle=-30,
                opacity=0.1,
                font=dict(color="white", size=50),
                xref="paper",
                yref="paper",
                x=0.5,
                y=0.5,
                showarrow=False,
            )
        ]
    )
    pio.templates.default = "plotly_dark+draft"

# Apply the theme
set_plotly_theme()

# # Sample data
# data = {
#     'Id': 5445,
#     'DeviceId': 1229,
#     'TagId': 1133749,
#     'Name': 'Acceleration Axis 1',
#     'XUnits': 'us',
#     'YUnits': 'g',
#     'Timestamp': '08/13/2024 14:19:27',
#     'XData': [0.0, 70.0, 140.0, 210.0, 280.0, 350.0, 420.0, 490.0, 560.0, 630.0, 700.0, 770.0, 840.0, 910.0, 980.0, 1050.0, 1120.0, 1190.0, 1260.0, 1330.0],
#     'YData': [0.0, -0.011, -0.005, -0.002, -0.011, 0.004, -0.01, 0.004, 0.009, 0.002, 0.0, 0.009, 0.011, 0.011, 0.006, -0.013, 0.0, 0.024, -0.007, 0.003],
#     'AccelData': [0.0, -0.011, -0.005, -0.002, -0.011, 0.004, -0.01, 0.004, 0.009, 0.002, 0.0, 0.009, 0.011, 0.011, 0.006, -0.013, 0.0, 0.024, -0.007, 0.003],
#     'AccelFFTData': [0.0, -1.3877787807814457e-17, 0.0, 0.12870071277513567, -0.04939789729593873, 0.08220168714228775, 0.044456783382145065, -0.14481091272204477, 0.05287116673790068, -0.09235569520951073, 0.05099294926813616, -0.10051656969077864, 0.05943620226221187, -0.04275171156685877, 0.07610961273142458, 0.009087681379710227, -0.06297350644756936, -0.13503768632250868, 0.016838968206379167, -0.02229204333797789],
#     'VelocityData': [0.0, -8.321060917133147e-05, -0.00020078224209194113, -0.0003470951037673099, -0.0005041740820957892, -0.0006448786712984455, -0.0007395691139855917, -0.0007643613864327523, -0.0007083788845234045, -0.0005775545712834955, -0.0003935365401041224, -0.00018780421849266148, 7.297007887417471e-06, 0.00016779839423203708, 0.00028463120097877153, 0.0003652509150282375, 0.0004297670382709364, 0.0005028208990737927, 0.0006039242418681941, 0.0007395593353316027],
#     'VelocityFFTData': [0.0, 0.0158120115308107, 0.0, 0.0543943099923329, 0.14171830887831122, -0.024473023917372272, 0.04525122382762216, -0.01939850123616127, -0.05313131604176449, -0.014027101299403216, -0.025405133667962244, -0.013073815336722111, -0.022110010740936832, -0.013943417484705331, -0.007832190194764891, 0.009882265991030419, 0.0014261058292950818, -0.002310435612229839, -0.018528206458306162, -0.009397102857094667]
# }

# Color palette
COLORS = {
    'accel_waveform': '#0D92F4',
    'accel_fft': '#77CDFF',
    'vel_waveform': '#C62E2E',
    'vel_fft': '#F95454'
}

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Acceleration vs Time',
        'Acceleration FFT',
        'Velocity vs Time',
        'Velocity FFT'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Add acceleration time series
fig.add_trace(
    go.Scatter(
        x=data['XData'],
        y=data['AccelData'],
        name='Acceleration',
        mode='lines+markers',
        line=dict(color=COLORS['accel_waveform'])
    ),
    row=1, col=1
)

# Add acceleration FFT
fig.add_trace(
    go.Scatter(
        x=data['XData'],
        y=data['AccelFFTData'],
        name='Acceleration FFT',
        mode='lines+markers',
        line=dict(color=COLORS['accel_fft'])
    ),
    row=1, col=2
)

# Add velocity time series
fig.add_trace(
    go.Scatter(
        x=data['XData'],
        y=data['VelocityData'],
        name='Velocity',
        mode='lines+markers',
        line=dict(color=COLORS['vel_waveform'])
    ),
    row=2, col=1
)

# Add velocity FFT
fig.add_trace(
    go.Scatter(
        x=data['XData'],
        y=data['VelocityFFTData'],
        name='Velocity FFT',
        mode='lines+markers',
        line=dict(color=COLORS['vel_fft'])
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title=f"Sensor Data Analysis - Device {data['DeviceId']} - {data['Name']}",
    height=800,
    width=1200,
    showlegend=True,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

# Update axes labels
fig.update_xaxes(title_text=f"Time ({data['XUnits']})", row=1, col=1, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')
fig.update_xaxes(title_text="Frequency", row=1, col=2, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')
fig.update_xaxes(title_text=f"Time ({data['XUnits']})", row=2, col=1, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')
fig.update_xaxes(title_text="Frequency", row=2, col=2, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')

fig.update_yaxes(title_text=f"Acceleration ({data['YUnits']})", row=1, col=1, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')
fig.update_yaxes(title_text="Magnitude", row=1, col=2, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')
fig.update_yaxes(title_text="Velocity", row=2, col=1, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')
fig.update_yaxes(title_text="Magnitude", row=2, col=2, gridcolor='rgba(128,128,128,0.2)', zerolinecolor='rgba(128,128,128,0.2)')

# Display the plot
fig.show()